In [ ]:
from __future__ import print_function
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from tqdm import tqdm_notebook, tqdm, trange
print(tf.__version__)

In [1]:
tf.set_random_seed(2)

NameError: name 'tf' is not defined

In [ ]:
(x_train, y_train), (x_test , y_test) = tf.keras.datasets.mnist.load_data()
x_train = (x_train/255).astype(np.float32)
x_test = (x_test/255).astype(np.float32)

val_range = int(len(x_test) * 0.8)

x_val = x_test[:val_range, :]
y_val = y_test[:val_range]

x_test = x_test[val_range:, :]
y_test = y_test[val_range:]

x_train = x_train.reshape((-1, 28, 28, 1))
x_val = x_val.reshape((-1, 28, 28, 1))
x_test = x_test.reshape((-1, 28, 28, 1))

In [ ]:
class CNNClassifier(keras.Model):
    def __init__(self, num_classes=10):
        super(CNNClassifier, self).__init__()
        self._layer1 = keras.layers.Conv2D(64, (5, 5), padding='same', strides=(2, 2), activation='relu',
                                                kernel_initializer='he_normal')
        self._layer2 = keras.layers.MaxPool2D()
        self._layer3 = keras.layers.Conv2D(32, (5, 5), padding='same', strides=(2, 2), activation='relu',
                                                kernel_initializer='he_normal')
        
        self._layer4 = keras.layers.GlobalAveragePooling2D()                
        self._layer5 = keras.layers.Dense(10, activation='softmax')

    def call(self, inputs):
        out = self._layer1(inputs)
        out = self._layer2(out)
        out = self._layer3(out)
        out = self._layer4(out)
        out = self._layer5(out)
        return out

In [ ]:
# Loss function
def loss_fn(model, x, y):
    logits = model(x)
    y = tf.dtypes.cast(y, tf.int32)
    return tf.losses.sparse_softmax_cross_entropy(labels=y, logits=logits)

In [ ]:
# Hyper Parameters
epoch = 100
batch_size = 100
learning_rate = 0.005
total_step = int(x_train.shape[0] / batch_size)

In [ ]:
# train
train_dataset = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_dataset = train_dataset.shuffle(buffer_size=10000)
train_dataset = train_dataset.batch(batch_size=batch_size)
print(train_dataset)

# validation
val_dataset = tf.data.Dataset.from_tensor_slices((x_val, y_val))
val_dataset = val_dataset.shuffle(buffer_size=10000)
val_dataset = val_dataset.batch(batch_size=batch_size)
print(val_dataset)

# test
test_dataset = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_dataset = test_dataset.shuffle(buffer_size=10000)
test_dataset = test_dataset.batch(batch_size=batch_size)
print(test_dataset)

In [ ]:
global_step = tf.train.get_or_create_global_step()

# creating optimizer
opt = tf.train.GradientDescentOptimizer(learning_rate=learning_rate)

# create writer for tensorboard
summary_writer = tf.contrib.summary.create_file_writer(logdir='./graphs/keras_eager/')

In [ ]:
mnist = CNNClassifier()

In [ ]:
# training
train_loss_hist = []
val_loss_hist = []

total_bar_length = tqdm_notebook(range(epoch))

device = '\cpu:0' if tfe.num_gpus() == 0 else 'gpu:0'

with tf.device(device):
    for i in total_bar_length:

        avg_train_loss = 0
        avg_val_loss = 0
        train_step = 0
        val_step = 0

        # training
        with summary_writer.as_default(), tf.contrib.summary.always_record_summaries():
            for x, y in train_dataset:
                with tf.GradientTape() as tape:
                    train_loss = loss_fn(mnist, x, y)
                grads = tape.gradient(target=train_loss, sources=mnist.variables)
                opt.apply_gradients(grads_and_vars=zip(grads, mnist.variables), global_step=global_step)
                tf.contrib.summary.scalar(name='train_loss', tensor=train_loss)
                avg_train_loss += train_loss
                train_step += 1
            else:
                avg_train_loss /= train_step
                train_loss_hist.append(avg_train_loss)

            # validation
            for _x, _y in val_dataset:
                val_loss = loss_fn(mnist, _x, _y)
                tf.contrib.summary.scalar(name='val_loss', tensor=val_loss)
                avg_val_loss += val_loss
                val_step += 1
            else:
                avg_val_loss /= val_step
                val_loss_hist.append(avg_val_loss)

        if (i + 1) % 10 == 0:
            print("epoch : {:3}, train_loss : {:.2f}, val_loss : {:.2f}".format(i, avg_train_loss, avg_val_loss))

In [ ]:
plt.plot(train_loss_hist, label="train")
plt.plot(val_loss_hist, label="validation")
plt.legend()
plt.show()

In [ ]:
scores = list()
yhat = list()
for __x, __y in test_dataset:
    out = mnist.call(__x)
    score = np.argmax(out, axis=1)
    
    scores.extend(score.tolist())
    yhat.extend(__y.numpy().tolist())

scores = np.asarray(scores)
yhat = np.asarray(yhat)

print("acc : {:.2%}".format(np.mean(yhat == scores)))